<a href="https://colab.research.google.com/github/SIDIBEMoussa/Texte_mining-and-web-mining/blob/main/Conception_KMeans_fo_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Nom: SIDIBE</h2>
<h1>Prénom:Moussa</h1>
<h2>Filière: Data Science</h2>

<h1> <center>Examen de Web et Text Mining </center></h1>

<h2> 1) Créez une base de 15 textes à partir du web avec leurs classes respectives :</h2>
<h2>(5 textes de sport, 5 textes d’économie, et 5 textes de politique) </h2>

In [1]:
BE=[["le club de football londonien est qualifié","sport"],
    ["le défenseur a marqué le dernier but","sport"],
    ["aucun club anglais n'a encaissé de but","sport"],
    ["le maroc est qualifié","sport"],
    ["le brésil est qualifié","sport"],
    ["le maroc a redressé son économie","économie"],
    ["la covid a touché l'économie marocaine","économie"],
    ["plusieurs entreprises sont en faillite","économie"],
    ["le PIB est en baisse","économie"],
    ["la bourse a fermé sur une baisse","économie"],
    ["le gouvernement envisage un nouveau confinement","politique"],
    ["le parlement a validé la loi 36.30","politique"],
    ["le gouvernement est parvenu à un accord","politique"],
    ["le Roi a promulgué la nouvelle loi","politique"],
    ["la convention internationnale","politique"]]

<h2> 2) Créez une fonction qui crée une liste L de tous les mots figurant dans cette base (sans répétition)</h2>

In [2]:
#création du VA
def texte_to_list_unique(BE):
    VA=[]
    for i in range(len(BE)):
        for j in range(len(BE[i][0].split())):
            if not BE[i][0].split()[j] in VA:
                VA.append(BE[i][0].split()[j])
    return VA

In [3]:
L=texte_to_list_unique(BE)

<h2> 3) Créez une première fonction T2V qui transforme chaque texte T en un vecteur d’occurrences des mots de la liste L</h2>

In [4]:
#convertir texte en vect num
def T2V(t,L):
    VN=[]
    T=t
    if type(T)!=list:
        T=T.split()
    for j in range(len(L)):
        cmpt=0
        for i in range(len(T)):
            if T[i]==L[j]:
                cmpt=cmpt+1
        VN.append(cmpt)    
    return VN

<h2> 4) Créez une liste de mots vides MV (pronoms, prépositions, conjonctions) et utilisez la pour réduite la dimension de L.</h2> <h2>Quel est le taux de réduction obtenu ?</h2>

In [5]:
MV=["a","à","au","ce","ces","ça","ceux","celle","du","de","des","dans","le","la","les","et","est","en","un","une","sur","son","sa","se","pour","avec"]

In [6]:
def remove_in_mv(L,MV):
    if type(L)!=list:
        L=L.split()
    total=len(L)
    nbre_supprime=0
    for word in MV:
        if word in L:
            L.remove(word)
            nbre_supprime+=1
    taux_reduction=nbre_supprime/total
    print("Le taux de réduction après suppression des (pronoms, prépositions, conjonctions) est:{:.2%}".format(taux_reduction))
    return L

In [7]:
L=remove_in_mv(L,MV)

Le taux de réduction après suppression des (pronoms, prépositions, conjonctions) est:20.37%


<h2> 5) Créez une fonction ELIM qui élimine les mots de la liste L ayant figuré dans les 3 classes et garde uniquement ceux qui figurent dans une ou deux classes.</h2> <h2> Quel est le taux de réduction obtenu ? </h2>

In [8]:
def ELIM(L):
    if type(L)!=list:
        L=L.split()
    nbre_mot_supp=0
    nbre_mot_total=len(L)
    for word in L:
        classe=[]
        for i in range(len(BE)):
            if word in BE[i][0] and BE[i][1] not in classe:
                classe.append(BE[i][1])
        if len(classe)>=3:
            L.remove(word)
            nbre_mot_supp+=1
    taux_reduction=nbre_mot_supp/nbre_mot_total
    print("Le taux de réduction après suppression des mots apparaissant dans plus de deux classes est:{:.2%}".format(taux_reduction))       
    return L

In [9]:
L=ELIM(L)

Le taux de réduction après suppression des mots apparaissant dans plus de deux classes est:0.00%


<h2> 6) Créez une fonction DIST qui calcule la distance euclidienne entre deux vecteurs </h2>

In [10]:
from math import sqrt
#Cette distance prend en compte la différence des vectuers X et Y
def DIST(X,Y):
    d=0
    rang=max(len(X),len(Y))
    for i in range(rang):
        if i>len(Y)-1:
            break
        else:
            d=d+(X[i]-Y[i])**2
    return sqrt(d)

In [23]:
#calculer les distances entre le nouveau et la base:
import pandas as pd
from simple_colors import *
def KNN(K,BNum,TNum):
    Label=[]
    distanc=[]
    for tab in BNum:
        distanc.append(DIST(tab[0],TNum))
        Label.append(tab[1])

    df=pd.DataFrame({"Distance":distanc,"Label":Label})
        
    df=df.sort_values("Distance")
    #Nous conservons juste les classes candidates
    decision_fields=df[:K]
    count_occu_class={classe:list(decision_fields.Label).count(classe) for classe in set(decision_fields.Label)}
    print(count_occu_class)
    class_predite=sorted(count_occu_class.items(), key=lambda t: t[1])[-1][0]
    print(green("La classe du nouveau texte est le :{}".format(class_predite),"bold"))
    
    return df


In [24]:
#Convertir la base et le nouveau en numérique
BNum=[]
for i in range(len(BE)):
    BNum.append([T2V(BE[i][0],L),BE[i][1]])

<h2> 7) Proposez un texte pour lequel : (en utilisant T2V)  </h2>
<h3> Parmi les 15 textes de la base, la DIST minimale correspond à la classe économie.</h3>

In [25]:
T2="le PIB chute et la bourse baisse"

In [26]:
TNum=T2V(T2,L)

In [27]:
df=KNN(3,BNum,TNum)

{'sport': 1, 'économie': 2}
La classe du nouveau texte est le :économie


<h2> 8) Proposez un texte pour lequel : (en utilisant T2V + MV + ELIM)</h2>
<h3> Parmi les 15 textes de la base, la DIST minimale correspond à la classe sport. </h3>

In [28]:
N="le défenseur français a marqué avec son club"

In [29]:
N=remove_in_mv(N,MV)
N=ELIM(N)
N=T2V(N,L)

Le taux de réduction après suppression des (pronoms, prépositions, conjonctions) est:50.00%
Le taux de réduction après suppression des mots apparaissant dans plus de deux classes est:0.00%


In [30]:
KNN(3,BNum,N)

{'sport': 3}
La classe du nouveau texte est le :sport


,Distance,Label
1,1.732051,sport
0,2.236068,sport
3,2.236068,sport
4,2.236068,sport
8,2.236068,économie
14,2.236068,politique
5,2.449490,économie
9,2.449490,économie
12,2.449490,politique
2,2.645751,sport


<h2> 9) Proposez un texte pour lequel :</h2>
<h3> La classe de la DIST minimale (en utilisant T2V) est différente de la classe de la DIST minimale (en utilisant T2V + MV + ELIM) </h3>

In [31]:
T1="convention du Roi a promulgué la nouvelle loi qualifié"

In [32]:
N1=remove_in_mv(T1,MV)
N1=ELIM(N1)
N1=T2V(N1,L)

Le taux de réduction après suppression des (pronoms, prépositions, conjonctions) est:33.33%
Le taux de réduction après suppression des mots apparaissant dans plus de deux classes est:0.00%


In [33]:
KNN(3,BNum,N1)

{'sport': 2, 'politique': 1}
La classe du nouveau texte est le :sport


,Distance,Label
13,1.414214,politique
3,2.449490,sport
4,2.449490,sport
14,2.449490,politique
0,2.828427,sport
8,2.828427,économie
11,2.828427,politique
5,3.000000,économie
9,3.000000,économie
12,3.000000,politique


In [34]:
N2=T2V(T1,L)

In [35]:
KNN(3,BNum,N2)

{'sport': 2, 'politique': 1}
La classe du nouveau texte est le :sport


,Distance,Label
13,1.414214,politique
3,2.449490,sport
4,2.449490,sport
14,2.449490,politique
0,2.828427,sport
8,2.828427,économie
11,2.828427,politique
5,3.000000,économie
9,3.000000,économie
12,3.000000,politique


<h2> 10) Quel modèle d’apprentissage artificiel avez-vous utilisé dans les questions 7 à 9 ? </h2>
<h3>Donnez 1 avantage et 1 inconvénient de ce modèle.</h3>

Dans 7 à 9 j'ai utilisé la méthode du plus proche voisin avec le paramètre K permettant de spécifier le nombre de support sur
lequel la décision sera prise et ce nombre doit être impair pour éviter la parité lors de la vote des classes.

<h4>Avantages:</h4>
Cet algorithme est plus facile à comprendre et à implementer
<h4>Inconvénients:</h4>
L'algorithme devient beaucoup plus lent à mesure que le nombre d'exemples d'apprentissage augmente et nécéssite.
Cet algorithme nécéssite une base de donnée assez très conséquent pour permettre une précision assez conséquente. Nous avons expérimenté cet aspect notamment pour les nouveaux hors de la base de nouveaux. Par conséquent cet algorithme peut être sujet de surapprentissage très rapidement.